In [7]:
!pip install -q TimeseriesGenerator

ERROR: Could not find a version that satisfies the requirement TimeseriesGenerator (from versions: none)
ERROR: No matching distribution found for TimeseriesGenerator


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator

from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras import Sequential

#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")


In [31]:
from pandas_datareader import data
from google.colab import files
uploaded = files.upload()

Saving AirPassengers.csv to AirPassengers (1).csv


In [32]:
import io
df = pd.read_csv(io.BytesIO(uploaded['AirPassengers.csv']))
print(df,sep=',')
df.head()

       Month  #Passengers
0    1949-01          112
1    1949-02          118
2    1949-03          132
3    1949-04          129
4    1949-05          121
..       ...          ...
139  1960-08          606
140  1960-09          508
141  1960-10          461
142  1960-11          390
143  1960-12          432

[144 rows x 2 columns]


,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [33]:
df.Month = pd.to_datetime(df.Month)
df = df.set_index('Month')
df.head()

,#Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [0]:
train, test = df[:-12], df[-12:]

In [0]:
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test= scaler.transform(test)

In [36]:
n_input = 12
n_features = 1

generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)

model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit_generator(generator, epochs=180)


ValueError: ignored

In [0]:
pred_list = []

batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):
  pred_list.append(model.predict(batch)[0])
  batch = np.append(batch[:, 1:, :], [[pred_list[i]]], axis=1)

In [0]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=df[-n_input:].index, columns = ['predictions'])

df_test = pd.concat([df, df_predict], axis=1)

In [0]:
df_test.tail()

In [0]:
plt.figure(figsize=(20,5))
plt.plot(df_test.index, df_test['AirPassengers'])
plt.plot(df_test.index, df_test['Predictions'], color='r')
plt.show()